In [18]:
import json
from yaml import safe_load
import pandas as pd
# Extractive
from nltk.tokenize import WordPunctTokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.reduction import ReductionSummarizer
from sumy.utils import get_stop_words
# Abstractive
# from sklearn.model_selection import train_test_split
# from keras_text_summarization.library.utility.plot_utils import plot_and_save_history
# from keras_text_summarization.library.seq2seq import Seq2SeqSummarizer
# from keras_text_summarization.library.applications.fake_news_loader import fit_text
import numpy as np

import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import glob

from html.parser import HTMLParser
from bs4 import BeautifulSoup
from pprint import pprint

In [24]:
import pickle

In [62]:
objects = []
with (open(
        "../../../Downloads/XSum-master/XSum-Dataset/xsum-extracts-from-downloads/10172644.data",
        "r")) as openfile:
    x = openfile.read()

x = x.split("\n\n")[1:]

x_dict = {"Summary": " ".join(x[0].split("\n")[1:]), 
         "Paragraphs": x[1].split("\n")[1:]}

In [75]:
def prepare_data(file):
    with open(file, 'r') as f:
        x = f.read().split("\n\n")[1:]
    return ({
        "Summary":
        " ".join(x[0].split("\n")[1:]),
        "Paragraphs":
        [el for el in x[1].split("\n")[1:] if el not in remove_words_list]
    })

In [80]:
remove_words_list = ['Share this with',
  'Email',
  'Facebook',
  'Messenger',
  'Messenger',
  'Twitter',
  'Pinterest',
  'WhatsApp',
  'LinkedIn',
  'Linkedin',                     
  'Copy this link',
  'These are external links and will open in a new window']

In [81]:
list_xsum = [
    prepare_data(file) for file in glob.glob("./datasets/XSUM_DATA/*.data")
]

In [88]:
with open("")

14143

In [42]:
print("\n\t\t SUMMARIZATION REVIEW\t\t\n")
print('[INFO] Loading configuration')
with open("./config.yml", 'r') as file:
    config_var = safe_load(file)["main"]

print("[INFO] Loading json data from")
with open(
        str(config_var['dataset_folder'])+"/"+str(config_var['data_to_use']),
        'r'
        ) as file:
    data = json.load(file)


		 SUMMARIZATION REVIEW		

[INFO] Loading configuration
[INFO] Loading json data from


In [48]:
data[0].keys()

dict_keys(['_id', 'Title', 'Date', 'Link', 'Summary', 'Paragraphs', 'Paragraphs Title'])

In [45]:
data[0]["Paragraphs"]

['Obama takes the gloves off for Biden, and Congress grills tech executives. It’s Thursday, and this is your politics tip sheet. Sign up here to get On Politics in your inbox every weekday.',
 'Mark Zuckerberg, Facebook’s chief executive, during his testimony yesterday. One lawmaker confronted him with Zuckerberg’s own emails, saying they showed a plot to take out a young competitor.',
 'With Trump lagging behind Biden in most national surveys, the Trump campaign has recently begun to advance the argument that public pollsters are not reaching enough Republican participants.',
 'The campaign has pointed to the fact that national polls now often find no more than a quarter of the electorate self-identifying as Republicans. The number was closer to one-third in 2016, according to exit polls and other postelection analyses.',
 '“National polls are often askew from what these 2016 exit polls were,” Bill Stepien, Trump’s campaign manager, argued in a call with reporters last week.',
 'Certa

In [10]:
data = pd.DataFrame(data)

In [11]:
print("[INFO] Removing articles without summary or paragraphs")
print("[INFO] Size before cleaning:", len(data))
data = data[(data["Summary"].map(len) >= 1) &
            (data["Paragraphs"].map(len) >= 1)]
print("[INFO] Size after cleaning:", len(data))

[INFO] Removing articles without summary or paragraphs
[INFO] Size before cleaning: 10335
[INFO] Size after cleaning: 9480


In [43]:
print("[INFO] Joining paragraphs")
data["Text"] = data["Paragraphs"].apply(lambda x: " ".join(x))

[INFO] Joining paragraphs


In [50]:
print("[INFO] Tokenization of Articles and Summaries.")
data["Tokenized_Paragraphs"] = (data["Text"].
                                apply(lambda x: x.lower()).
                                apply(WordPunctTokenizer().tokenize)
                               )

[INFO] Tokenization of Articles and Summaries.


# Sumy

Per poter fare summarization, esiste questa bellissima libreria chiamata `sumy`, che permette sia di definire una classe Documento, divisa in paragrafi, sia di usare le più importanti tecniche estrattive di *summarization*. 

Come punto di partenza, può essere qualcosa di interessante. È anche facile da implementare a partire dal mio dataset.

**COSA DA FARE**: contare in media di quante frasi sono fatte le summary, al fine di decidere quanto lunghe farle.

In [61]:
print("\n\t\t SUMMARIZATION REVIEW\t\t\n")
print('[INFO] Loading configuration')
with open("./config.yml", 'r') as file:
    config_var = safe_load(file)["main"]

print("[INFO] Loading json data from")
with open(
        str(config_var['dataset_folder'])+"/"+str(config_var['data_to_use']),
        'r'
        ) as file:
    data = pd.DataFrame(json.load(file))
    
print("[INFO] Removing articles without summary or paragraphs")
print("[INFO] Size before cleaning:", len(data))
data = data[(data["Summary"].map(len) >= 1) &
            (data["Paragraphs"].map(len) >= 1)]
print("[INFO] Size after cleaning:", len(data))    


		 SUMMARIZATION REVIEW		

[INFO] Loading configuration
[INFO] Loading json data from
[INFO] Removing articles without summary or paragraphs
[INFO] Size before cleaning: 10335
[INFO] Size after cleaning: 9480


In [104]:
data["Paragraphs_as_string"] = data["Paragraphs"].apply(
    lambda x: "\n\n".join(x))

In [105]:
parser = PlaintextParser.from_string(data["Paragraphs_as_string"][1], 
                                     tokenizer = Tokenizer('english'))

In [83]:
stemmer = Stemmer('english')

In [84]:
summarizer = LsaSummarizer(stemmer)

summarizer.stop_words = get_stop_words('english')

In [86]:
 for sentence in summarizer(parser.document, 2):
        print(sentence)

A poll late last week from The Des Moines Register showed her support at 17 percent, narrowly trailing Senator Bernie Sanders of Vermont, her ideological rival, but slightly ahead of former Vice President Joseph R. Biden Jr. and Mr. Buttigieg, the former mayor of South Bend, Ind.
“I am ready to poke my eyes out about whether to support the candidate most likely to be a moderate and win the election,” said Dea Epley Birtwistle, 62, a school social worker who attended Ms. Warren’s event in Mason City.


In [93]:
summarizer_2 = LuhnSummarizer(stemmer)

summarizer_2.stop_words = get_stop_words('english')
for sentence in summarizer_2(parser.document, 2):
    print(sentence)

MASON CITY, Iowa — Kristen Marttila’s alarm went off at 6 a.m. on Saturday and soon she was filling up two thermoses — one with hot water, one with soup — to gird for the day ahead: a two-hour drive to Mason City to campaign for Senator Elizabeth Warren in the icy cold.
A poll late last week from The Des Moines Register showed her support at 17 percent, narrowly trailing Senator Bernie Sanders of Vermont, her ideological rival, but slightly ahead of former Vice President Joseph R. Biden Jr. and Mr. Buttigieg, the former mayor of South Bend, Ind.


In [95]:
summarizer_3 = SumBasicSummarizer(stemmer)

summarizer_3.stop_words = get_stop_words('english')
for sentence in summarizer_3(parser.document, 2):
    print(sentence)

Ms. Warren would hear it herself at events.
This, to me, is what Iowa is all about.”


In [89]:
data["Summary"][1]

'In the early fall, Ms. Warren seemed to be the Democratic candidate to beat in Iowa. But many voters have had second thoughts over how her sweeping agenda would sell against President Trump.'

In [97]:
summarizer_4 = LexRankSummarizer(stemmer)

summarizer_4.stop_words = get_stop_words('english')
for sentence in summarizer_4(parser.document, 2):
    print(sentence)

Ms. Warren’s supporters believe her early investment in an Iowa field organization will help her at the finish.
This, to me, is what Iowa is all about.”


In [99]:
summarizer_5 = KLSummarizer(stemmer)

summarizer_5.stop_words = get_stop_words('english')
for sentence in summarizer_5(parser.document, 2):
    print(sentence)

Ms. Warren covered little new ground that day, talking about “running from the heart” and how “fear doesn’t win.” But on Tuesday night, before a national debate audience and with time running short to reignite her candidacy, Ms. Warren delivered her most emphatic answer to date about her electability: She cited her own electoral success compared to her male opponents, the political gains made by other female candidates and her determination to unify the Democrats.
A poll late last week from The Des Moines Register showed her support at 17 percent, narrowly trailing Senator Bernie Sanders of Vermont, her ideological rival, but slightly ahead of former Vice President Joseph R. Biden Jr. and Mr. Buttigieg, the former mayor of South Bend, Ind.


In [103]:
summarizer_6 = ReductionSummarizer(stemmer)

summarizer_6.stop_words = get_stop_words('english')
for sentence in summarizer_6(parser.document, 2):
    print(sentence)

Ms. Warren covered little new ground that day, talking about “running from the heart” and how “fear doesn’t win.” But on Tuesday night, before a national debate audience and with time running short to reignite her candidacy, Ms. Warren delivered her most emphatic answer to date about her electability: She cited her own electoral success compared to her male opponents, the political gains made by other female candidates and her determination to unify the Democrats.
“On the ground, based on what I’ve seen, Warren’s campaign is very focused on the person-to-person type of campaigning,” said Sandy Dockendorff, a former Democratic chairwoman in Des Moines County who has endorsed Ms. Warren.


# Encoder-decoder

Sebbene non abbia ancora provato delle tecniche di valutazione per gli algoritmi precedenti, voglio provare a vedere se un'architettura neurale potrebbe aiutare.

In particolare, di seguito, al posto che usare delle tecniche di *extractive summarization*, si va ad applicare dell'*abstractive summarization*.

Per questa libreria, i dati devono essere messi come unico file di testo. Inoltre, dato che usiamo *supervised learning*, si devono prendere come input le summary.

In [44]:
print("\n\t\t SUMMARIZATION REVIEW\t\t\n")
print('[INFO] Loading configuration')
with open("./config.yml", 'r') as file:
    config_var = safe_load(file)["main"]

print("[INFO] Loading json data from")
with open(
        str(config_var['dataset_folder'])+"/"+str(config_var['data_to_use']),
        'r'
        ) as file:
    data = pd.DataFrame(json.load(file))

print("[INFO] Removing articles without summary or paragraphs")
print("[INFO] Size before cleaning:", len(data))
data = data[(data["Summary"].map(len) >= 1) &
            (data["Paragraphs"].map(len) >= 1)].reset_index()
print("[INFO] Size after cleaning:", len(data))

print("[INFO] Joining paragraphs")
data["Text"] = data["Paragraphs"].apply(lambda x: " ".join(x))


		 SUMMARIZATION REVIEW		

[INFO] Loading configuration
[INFO] Loading json data from
[INFO] Removing articles without summary or paragraphs
[INFO] Size before cleaning: 10335
[INFO] Size after cleaning: 9480
[INFO] Joining paragraphs


In [65]:
# prepare data for training
data["Text_tokenized"] = data["Text"].apply(sent_tokenize)

data["Text_processed"] = (data["Text_tokenized"].
                          apply(lambda x: "\n".join(x)))

# do the same, but on the summaries
data["Summary_tokenized"] = data["Summary"].apply(sent_tokenize)

data["Summary_processed"] = (data["Summary_tokenized"].
                          apply(lambda x: "\n".join(x)))



In [74]:
X = data["Text_processed"]
Y = data.Summary_processed

In [78]:
print('loading csv file ...')
df = pd.read_csv("./keras-text-summarization-master/demo/data/fake_or_real_news.csv")

print('extract configuration from input texts ...')
Y = df.title
X = df['text']

loading csv file ...
extract configuration from input texts ...


In [79]:
config = fit_text(X, Y)

summarizer = Seq2SeqSummarizer(config)

In [81]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,
                                                Y,
                                                test_size=0.2,
                                                random_state=42)

In [84]:
history = summarizer.fit(Xtrain, Ytrain, Xtest, Ytest, epochs=10)

(5068,)
(1267,)
(5068, 500)
(1267, 500)
Epoch 1/10
79/79 [==============================] - 35s 443ms/step - loss: 1.1073 - accuracy: 0.0200 - val_loss: 1.0916 - val_accuracy: 0.0200
Epoch 2/10
10/79 [==>...........................] - ETA: 30s - loss: 1.1145 - accuracy: 0.0200

KeyboardInterrupt: 

Per qualche motivo, non riesce ad andare oltre il risultato che ottiene dopo la terza epoca. Lo addestro dunque poco, e vedo un attimo che coso riesce a sparare fuori.

In [83]:
for i in range(20):
    x = X[i]
    actual_headline = Y[i]
    headline = summarizer.summarize(x)
#     print('Article: ', x)
    print('Generated Summary: ', headline)
    print('Original Summary: ', actual_headline)


Generated Summary:  
Original Summary:  You Can Smell Hillary’s Fear
Generated Summary:  
Original Summary:  Watch The Exact Moment Paul Ryan Committed Political Suicide At A Trump Rally (VIDEO)
Generated Summary:  
Original Summary:  Kerry to go to Paris in gesture of sympathy
Generated Summary:  
Original Summary:  Bernie supporters on Twitter erupt in anger against the DNC: 'We tried to warn you!'
Generated Summary:  
Original Summary:  The Battle of New York: Why This Primary Matters
Generated Summary:  
Original Summary:  Tehran, USA
Generated Summary:  
Original Summary:  Girl Horrified At What She Watches Boyfriend Do After He Left FaceTime On
Generated Summary:  
Original Summary:  ‘Britain’s Schindler’ Dies at 106
Generated Summary:  
Original Summary:  Fact check: Trump and Clinton at the 'commander-in-chief' forum
Generated Summary:  
Original Summary:  Iran reportedly makes new push for uranium concessions in nuclear talks
Generated Summary:  
Original Summary:  With all th

In [86]:
data_dir_path = './keras-text-summarization-master/demo/data' # refers to the demo/data folder
model_dir_path = './keras-text-summarization-master/demo/models' # refers to the demo/models folder

# Another Attention Encoder-Decoer

Per qualche cazzo di motivo, quello di prima non riesce a funzionare - va a prevedere con la parole più frequente.

In [3]:
# Import Libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np  
import pandas as pd 
import re      
# To list all the available datasets currently present in the repo.
builders = tfds.list_builders()
data, info = tfds.load("cnn_dailymail", with_info=True)
train_data, test_data = data['train'], data['test']
# print info of the dataset
print(info)

Dataset cnn_dailymail downloaded and prepared to /Users/leonardoalchieri/tensorflow_datasets/cnn_dailymail/3.1.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='cnn_dailymail',
    full_name='cnn_dailymail/3.1.0',
    description="""
    CNN/DailyMail non-anonymized summarization dataset.
    
    There are two features:
      - article: text of news article, used as the document to be summarized
      - highlights: joined text of highlights with <s> and </s> around each
        highlight, which is the target summary
    """,
    homepage='https://github.com/abisee/cnn-dailymail',
    data_path='/Users/leonardoalchieri/tensorflow_datasets/cnn_dailymail/3.1.0',
    download_size=558.32 MiB,
    dataset_size=1.27 GiB,
    features=FeaturesDict({
        'article': Text(shape=(), dtype=tf.string),
        'highlights': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=('article', 'highlights'),
    splits={
        'test': <SplitInfo num_examples=11490,